In [46]:
from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
import pandas as pd

In [2]:
!which chromedriver

/usr/local/bin/chromedriver


In [15]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [4]:
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'

browser.visit(url)


In [5]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

news_article = soup.find_all('div', class_='content_title')[1].a.text
news_p=soup.find('div', class_='article_teaser_body').text


In [6]:
url2="https://www.jpl.nasa.gov/spaceimages/"
browser.visit(url2)

In [7]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')



In [8]:
img_url = soup.find('article', class_='carousel_item')["style"].split("'")[1]
img_url

'/spaceimages/images/wallpaper/PIA18904-1920x1200.jpg'

In [9]:

featured_image_url="http://www.jpl.nasa.gov"+img_url



featured_image_url


'http://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA18904-1920x1200.jpg'

In [56]:
url = "https://twitter.com/marswxreport?lang=en"
browser.visit(url)

html = browser.html
weather_soup = BeautifulSoup(html, "html.parser")

    # First, find a tweet with the data-name `Mars Weather`


 

In [59]:
tweet_attrs = {"class": "tweet", "data-name": "Mars Weather"}
mars_weather_tweet = weather_soup.find("div", attrs=tweet_attrs)

    # Next, search within the tweet for the p tag containing the tweet text


In [61]:
weather_soup

<html dir="ltr" lang="en" style="font-size: 15px;"><head><meta charset="utf-8"/>
<meta content="width=device-width,initial-scale=1,maximum-scale=1,user-scalable=0,viewport-fit=cover" name="viewport"/>
<link href="//abs.twimg.com" rel="preconnect"/>
<link href="//api.twitter.com" rel="preconnect"/>
<link href="//pbs.twimg.com" rel="preconnect"/>
<link href="//t.co" rel="preconnect"/>
<link href="//video.twimg.com" rel="preconnect"/>
<link href="//abs.twimg.com" rel="dns-prefetch"/>
<link href="//api.twitter.com" rel="dns-prefetch"/>
<link href="//pbs.twimg.com" rel="dns-prefetch"/>
<link href="//t.co" rel="dns-prefetch"/>
<link href="//video.twimg.com" rel="dns-prefetch"/>
<link as="script" crossorigin="anonymous" href="https://abs.twimg.com/responsive-web/web/polyfills.1c57c6c5.js" nonce="" rel="preload"/>
<link as="script" crossorigin="anonymous" href="https://abs.twimg.com/responsive-web/web/vendors~main.cd037d05.js" nonce="" rel="preload"/>
<link as="script" crossorigin="anonymous" 

In [58]:
mars_weather = mars_weather_tweet.find("p", "tweet-text").get_text()

AttributeError: 'NoneType' object has no attribute 'find'

In [20]:
astro_url="https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
head_url=astro_url.split("search")[0]
astro=requests.get(astro_url)

In [22]:
soup_ast=BeautifulSoup(astro.text, "html.parser")

In [27]:
hemp_list=soup_ast.find_all('div', class_="item")

In [39]:
hemp_list[0].img["src"]

'/cache/images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png'

In [25]:
hemisphere_images_url =[]

In [43]:
hemp_list=soup_ast.find_all('div', class_="item")

# Loop through returned results
for hemp in hemp_list:
    # Error handling
    try:
        # Identify and return title of listing
         title= hemp.find('h3').text
        # Identify and return price of listing
         hem_imp= hemp.img["src"]
        # Identify and return link to listing
         image=head_url+img_url

        # Run only if title, price, and link are available
         if (title and image):
            # Print results
            print('-------------')
            print(title)
            print(image)
           


            # Dictionary to be inserted as a MongoDB document
            post = {
                'titles': title,
                'image_url': image
            }

            hemisphere_images_url.append(post)
    except Exception as e:
        print(e)

-------------
Cerberus Hemisphere Enhanced
https://astrogeology.usgs.gov//spaceimages/images/wallpaper/PIA18904-1920x1200.jpg
-------------
Schiaparelli Hemisphere Enhanced
https://astrogeology.usgs.gov//spaceimages/images/wallpaper/PIA18904-1920x1200.jpg
-------------
Syrtis Major Hemisphere Enhanced
https://astrogeology.usgs.gov//spaceimages/images/wallpaper/PIA18904-1920x1200.jpg
-------------
Valles Marineris Hemisphere Enhanced
https://astrogeology.usgs.gov//spaceimages/images/wallpaper/PIA18904-1920x1200.jpg


In [44]:
facts_url ="https://space-facts.com/mars/"

In [48]:
tables = pd.read_html(facts_url)
Mars_df = tables[2]

In [50]:
Mars_df

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers
